# Welcome!

This example shows a slightly more complicated training paradigm in MultiBench. Namely, we'll run L2-MCTN on MOSI.

This tutorial assumes you've followed along with the first tutorial, as we'll focus on the differences between this task and standard supervised learning in MultiBench.

To begin, let's clone the repo and setup our interpreter to run commands inside the folder.

In [2]:
!git clone https://github.com/pliang279/MultiBench.git
%cd MultiBench

Cloning into 'MultiBench'...
remote: Enumerating objects: 4890, done.
remote: Counting objects: 100% (1906/1906), done.
remote: Compressing objects: 100% (1018/1018), done.
remote: Total 4890 (delta 1289), reused 1370 (delta 885), pack-reused 2984
Receiving objects: 100% (4890/4890), 46.51 MiB | 32.22 MiB/s, done.
Resolving deltas: 100% (3347/3347), done.
/content/MultiBench/MultiBench


Try to download the data file for MOSI using the below command. If this does not work for you, please download the data file locally, and upload it to the folder "/content/MultiBench/"

In [3]:
!mkdir data
!mkdir temp
!pip install gdown && gdown https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU&export=download

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/u/0/uc?id=1KvKynJJca5tDtI5Mmp6CoRh9pQywH8Xp 



As Colab famously has bad handling of Conda env files, we'll install the dependencies manually so that it works. Please note that other systems might require installation of a long list of other dependencies.

In [8]:
!pip install memory-profiler

  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284 sha256=72f96169512ed301798c7fcc28fd951c8a7487425a6db3c1a75674d5d1f9b6df
  Stored in directory: /root/.cache/pip/wheels/67/2b/fb/326e30d638c538e69a5eb0aa47f4223d979f502bbdb403950f
Successfully built memory-profiler


From here, let's import some of MultiBench and get working. First, we'll import what is required from all MultiBench programs:

In [14]:
from torch import nn
import torch
import sys
import os

start training ---------->>
Train Epoch 0, total loss: 3.0611612796783447, regression loss: 1.3306723833084106, embedding loss: 1.730488896369934
Start Evaluating ---------->>
Eval Epoch: 0, MAE: 1.4072818756103516, Acc1: 0.5747663551401869, Acc2: 0.6119402985074627
<------------ Saving Best Model

start training ---------->>
Train Epoch 1, total loss: 3.0371196269989014, regression loss: 1.3246451616287231, embedding loss: 1.7124744653701782
Start Evaluating ---------->>
Eval Epoch: 1, MAE: 1.4050803184509277, Acc1: 0.5747663551401869, Acc2: 0.6119402985074627
<------------ Saving Best Model

start training ---------->>
Train Epoch 2, total loss: 3.0315959453582764, regression loss: 1.3075841665267944, embedding loss: 1.724011778831482
Start Evaluating ---------->>
Eval Epoch: 2, MAE: 1.4035106897354126, Acc1: 0.5747663551401869, Acc2: 0.6119402985074627
<------------ Saving Best Model

start training ---------->>
Train Epoch 3, total loss: 3.0435800552368164, regression loss: 1.32058

Then, let's import the dataloaders for MOSI, and import that data using the path we stored the MOSI_RAW.pkl file to.

In [ ]:
from datasets.affect.get_data import get_dataloader # noqa


traindata, validdata, testdata = \
    get_dataloader('/content/MultiBench/mosi_raw.pkl', robust_test=False)

Then, let's define the encoder and decoder modules for each modality, taken from the associated section of MultiBench.

In [ ]:
from unimodals.common_models import GRU, MLP # noqa
from fusions.MCTN import Encoder, Decoder # noqa


max_seq = 20
feature_dim = 300
hidden_dim = 32

encoder0 = Encoder(feature_dim, hidden_dim, n_layers=1, dropout=0.0).cuda()
decoder0 = Decoder(hidden_dim, feature_dim, n_layers=1, dropout=0.0).cuda()
encoder1 = Encoder(hidden_dim, hidden_dim, n_layers=1, dropout=0.0).cuda()
decoder1 = Decoder(hidden_dim, feature_dim, n_layers=1, dropout=0.0).cuda()

reg_encoder = nn.GRU(hidden_dim, 32).cuda()

Then, let's define the classification head for our model:

In [15]:
from unimodals.common_models import MLP # noqa
head = MLP(32, 64, 1).cuda()

Lastly, let's define the model training structure, and train our MCTN network. Here, it takes in not only encoders and decoders for each input modality, but also the reg_encoder:

In [ ]:
from private_test_scripts.all_in_one import all_in_one_train # noqa
from training_structures.MCTN_Level2 import train, test # noqa

allmodules = [encoder0, decoder0, encoder1, decoder1, reg_encoder, head]


def trainprocess():
    train(
        traindata, validdata,
        encoder0, decoder0, encoder1, decoder1,
        reg_encoder, head,
        criterion_t0=nn.MSELoss(), criterion_c=nn.MSELoss(),
        criterion_t1=nn.MSELoss(), criterion_r=nn.L1Loss(),
        max_seq_len=20,
        mu_t0=0.01, mu_c=0.01, mu_t1=0.01,
        dropout_p=0.15, early_stop=False, patience_num=15,
        lr=1e-4, weight_decay=0.01, op_type=torch.optim.AdamW,
        epoch=200, model_save='best_mctn.pt')


all_in_one_train(trainprocess, allmodules)

model = torch.load('best_mctn.pt').cuda()

test(model, testdata, 'mosi', no_robust=True)
